# Description

The Python `timeit` module provides very good quality timing of operations, with statistics, disabling of garbage collection, and options to run separate setup versus timing code.  Within Jupyter or IPython, the `%time` and `%timeit` magics offer similar bundled capability.

In this exercise, you will create something less sophisticated than that, but still useful.  You should develop a custom context manager called `Timer` that will run code multiple times, and save both the result from each run and the time it took.  For example, using the `slow_random_normal()` function provided in the Setup:

```python
>>> with Timer(slow_random_normal, loops=5) as fn:
...     fn.run(-3, stdev=5)

>>> print("Timers:", fn.timers)
Timers: [0.0501605, 0.0502064, 0.0501945, 0.0502957, 0.0501452]
>>> print("Results:", fn.results)
Results: [-0.2525025, -1.4776930, -2.4067729, -1.3618136, -0.6298354]
```

Note that the test cases assume `slow_random_normal()` retains the same implementation provided.

# Setup

In [1]:
from time import sleep
from random import random

def slow_random_normal(mean=0, stdev=1):
    sleep(0.05)
    r = mean + stdev*random()
    return r

class Timer:
    "Implementation here"

# Solution

In [2]:
from time import perf_counter

class Timer:
    def __init__(self, fn=lambda *args, **kws: None, loops=1):
        self.loops = loops
        self.fn = fn
        self.timers = list()
        self.results = list()
        
    def __enter__(self):
        return self
    
    def __exit__(self, type_, value, tb):
        pass
    
    def run(self, *args, **kws):
        for _ in range(self.loops):
            start = perf_counter()
            result = self.fn(*args, **kws)
            self.timers.append(perf_counter() - start)
            self.results.append(result)       

# Test Cases

In [3]:
def test_is_cm():
    assert hasattr(Timer, '__enter__')
    assert hasattr(Timer, '__exit__')
    
test_is_cm()

In [5]:
def test_is_timing():
    from math import isclose
    from statistics import mean
    with Timer(slow_random_normal, loops=100) as fn:
        fn.run(-5, stdev=0.1)
    assert len(fn.timers) == len(fn.results) == 100
    assert isclose(mean(fn.timers), 0.05, abs_tol=0.01)
    assert isclose(mean(fn.results), -5, abs_tol=0.1)
    
test_is_timing()